# 羽球分析系統 - 功能演示

此 Notebook 示範如何使用各個模組進行羽球影片分析。

1. 場地檢測
2. YOLO 物體檢測
3. 骨架追踪
4. 球軌跡追踪
5. OCR 比分識別
6. 完整分析流程
7. 影片統計分析


## 1. 環境設置

In [15]:
import sys
import os

# 添加 src 到路徑
sys.path.append('../src')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display

# 設定顯示選項
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 2. 場地檢測示範

In [16]:
from detection.court_detector import CourtDetector

# 載入測試影像
cap = cv2.VideoCapture('../data/videos/demo.mp4')
ret, frame = cap.read()
frame = cv2.resize(frame, (800, 450))

# 創建檢測器
detector = CourtDetector()

# 執行檢測
is_ok, corners, output, area_percent = detector.detect(frame)
output = detector.draw_court_status(output, is_ok)

# 顯示結果
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title(f'場地檢測結果{'成功' if is_ok else '失敗'} (面積: {area_percent:.2f}%)')
plt.axis('off')
plt.show()

cap.release()

SyntaxError: invalid syntax (3297033243.py, line 17)

## 3. YOLO 物體檢測示範

In [ ]:
from detection.yolo_detector import YOLODetector

# 創建檢測器
yolo = YOLODetector()

# 重新載入影像
cap = cv2.VideoCapture('../data/videos/demo.mp4')
ret, frame = cap.read()
frame = cv2.resize(frame, (800, 450))

# 執行檢測
labels, bbox, result = yolo.detect(frame)

# 繪製結果
output = yolo.draw_detections(frame, labels, bbox)

# 顯示
plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title(f'YOLO 檢測結果 - 檢測到 {len(labels)} 個物體')
plt.axis('off')
plt.show()

# 顯示檢測到的物體
for i, label_idx in enumerate(labels):
    label = yolo.classes[int(label_idx)]
    conf = bbox[i][4].item()
    print(f'物體 {i+1}: {label} (信心度: {conf:.2f})')

cap.release()

ModuleNotFoundError: No module named 'torch'

## 4. 骨架追踪示範

In [ ]:
from detection.skeleton_tracker import SkeletonTracker

# 創建追踪器
tracker = SkeletonTracker()

# 重新載入影像並取得球員區域
cap = cv2.VideoCapture('../data/videos/demo.mp4')
ret, frame = cap.read()
frame = cv2.resize(frame, (800, 450))

# 假設我們已經有球員的邊界框
player_bbox = yolo.get_player_bbox(labels, bbox, frame.shape)

if player_bbox:
    x1, y1, x2, y2 = player_bbox
    player_img = frame[y1:y2, x1:x2]
    
    # 處理骨架
    pos_list, results = tracker.process_player(player_img)
    
    # 繪製骨架
    output = tracker.draw_skeleton(frame, results, bbox=player_bbox)
    
    # 顯示
    plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    plt.title(f'骨架追踪結果 - 檢測到 {len(pos_list)} 個關鍵點')
    plt.axis('off')
    plt.show()
else:
    print('未檢測到球員')

cap.release()

## 5. 球軌跡追踪示範

In [ ]:
from detection.ball_tracker import BallTracker

# 創建追踪器
ball_tracker = BallTracker(buffer_size=32)

# 處理多幀以獲得軌跡
cap = cv2.VideoCapture('../data/videos/demo.mp4')
frame_count = 0
max_frames = 100

while frame_count < max_frames:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.resize(frame, (800, 450))
    
    # 檢測羽球
    labels, bbox, _ = yolo.detect(frame)
    shuttle_positions = yolo.get_shuttle_positions(labels, bbox, frame.shape)
    
    # 添加軌跡點
    for pos in shuttle_positions:
        ball_tracker.add_point(pos)
    
    frame_count += 1

# 繪製最後一幀的軌跡
output = ball_tracker.draw_trajectory(frame, color=(255, 0, 0))

plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
plt.title(f'球軌跡追踪 - {len(ball_tracker.get_trajectory_points())} 個軌跡點')
plt.axis('off')
plt.show()

cap.release()

# 顯示軌跡統計
trajectory = ball_tracker.get_trajectory_points()
if trajectory:
    x_coords = [p[0] for p in trajectory]
    y_coords = [p[1] for p in trajectory]
    
    plt.figure(figsize=(10, 6))
    plt.plot(x_coords, y_coords, 'b-o', markersize=3)
    plt.xlabel('X 座標')
    plt.ylabel('Y 座標')
    plt.title('球運動軌跡 (2D 視圖)')
    plt.grid(True)
    plt.gca().invert_yaxis()  # Y 軸反轉（影像座標系）
    plt.show()

## 6. OCR 比分識別示範

In [ ]:
from ocr.score_reader import ScoreReader

# 創建 OCR 讀取器
reader = ScoreReader()

# 載入原始解析度影像（OCR 需要高解析度）
cap = cv2.VideoCapture('../data/videos/demo.mp4')
ret, orig_frame = cap.read()

# 讀取比分和球員名稱
player_names, scores = reader.read_score_and_names(orig_frame)

print('=== OCR 識別結果 ===')
print('\n球員名稱:')
print(player_names)
print('\n比分:')
print(scores)

cap.release()

## 7. 完整分析流程

In [ ]:
# 這個 cell 示範如何組合所有模組進行完整分析

def analyze_frame(frame, orig_frame):
    """分析單幀影像"""
    results = {}
    
    # 1. 場地檢測
    is_ok, corners, _, area = detector.detect(frame)
    results['court_ok'] = is_ok
    results['court_area'] = area
    
    if is_ok:
        # 2. YOLO 檢測
        labels, bbox, _ = yolo.detect(frame)
        results['num_objects'] = len(labels)
        
        # 3. 羽球位置
        shuttle_pos = yolo.get_shuttle_positions(labels, bbox, frame.shape)
        results['shuttle_positions'] = shuttle_pos
        
        # 4. 球員骨架
        player_bbox = yolo.get_player_bbox(labels, bbox, frame.shape)
        if player_bbox:
            x1, y1, x2, y2 = player_bbox
            player_img = frame[y1:y2, x1:x2]
            pos_list, _ = tracker.process_player(player_img)
            results['skeleton_points'] = len(pos_list)
        
        # 5. OCR (僅在需要時執行，因為較慢)
        # player_names, scores = reader.read_score_and_names(orig_frame)
        # results['player_names'] = player_names
        # results['scores'] = scores
    
    return results

# 測試單幀
cap = cv2.VideoCapture('../data/videos/demo.mp4')
ret, frame = cap.read()
orig_frame = frame.copy()
frame = cv2.resize(frame, (800, 450))

results = analyze_frame(frame, orig_frame)

print('=== 分析結果 ===')
for key, value in results.items():
    print(f'{key}: {value}')

cap.release()

## 8. 影片處理範例

In [ ]:
# 處理影片的前 N 幀並收集統計資訊

cap = cv2.VideoCapture('../data/videos/demo.mp4')
stats = {
    'total_frames': 0,
    'court_detected': 0,
    'shuttles_detected': 0,
    'frames_with_shuttle': 0
}

max_frames = 300  # 只處理前 300 幀

while stats['total_frames'] < max_frames:
    ret, frame = cap.read()
    if not ret:
        break
    
    stats['total_frames'] += 1
    frame_resized = cv2.resize(frame, (800, 450))
    
    # 檢測場地
    is_ok, _, _, _ = detector.detect(frame_resized)
    if is_ok:
        stats['court_detected'] += 1
        
        # 檢測羽球
        labels, bbox, _ = yolo.detect(frame_resized)
        shuttle_pos = yolo.get_shuttle_positions(labels, bbox, frame_resized.shape)
        
        if shuttle_pos:
            stats['frames_with_shuttle'] += 1
            stats['shuttles_detected'] += len(shuttle_pos)
    
    # 顯示進度
    if stats['total_frames'] % 50 == 0:
        print(f"已處理 {stats['total_frames']} 幀...")

cap.release()

# 顯示統計結果
print('\n=== 影片分析統計 ===')
print(f"總幀數: {stats['total_frames']}")
print(f"場地檢測成功率: {stats['court_detected']/stats['total_frames']*100:.2f}%")
print(f"檢測到羽球的幀數: {stats['frames_with_shuttle']}")
print(f"平均每幀羽球數: {stats['shuttles_detected']/max(stats['frames_with_shuttle'], 1):.2f}")

# 繪製統計圖表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 場地檢測統計
labels_court = ['檢測成功', '檢測失敗']
values_court = [stats['court_detected'], stats['total_frames'] - stats['court_detected']]
ax1.pie(values_court, labels=labels_court, autopct='%1.1f%%')
ax1.set_title('場地檢測成功率')

# 羽球檢測統計
labels_shuttle = ['有羽球', '無羽球']
values_shuttle = [stats['frames_with_shuttle'], stats['total_frames'] - stats['frames_with_shuttle']]
ax2.pie(values_shuttle, labels=labels_shuttle, autopct='%1.1f%%')
ax2.set_title('羽球檢測率')

plt.tight_layout()
plt.show()